<a href="https://colab.research.google.com/github/flee2-trinity/Deep-Learning-in-Sports-Result-Prediction-/blob/main/SGTeams1213.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from google.colab import drive

drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/My Drive/2021 Fall/Thesis I'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import datetime

def findFiles(path): 
  return glob.glob(path)

In [ ]:
import csv

filename = '/content/gdrive/My Drive/2021 Fall/Thesis I/NBA Data/nba 12-13/NatStat-NBA2013-Team_Statlines.csv'
with open(filename, newline='') as f:
      reader = csv.reader(f)
      """
      print(list(reader))
      ['GameDay', 'GameID', 'TeamID', 'Team', 'OpponentID', 'Opponent', 'Location',
       'Division', 'Conference', 'Playoffs', 'WinOrLoss', 'PlayerType', 'MIN', 'PTS', 
       'FGM', 'FGA', '3FM', '3FA', 'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'OREB', 'TO', 'PF']
      """
      next(reader)
      rawDataL = list(reader)
print(rawDataL[0])

['2012-10-30', '28027', '9', 'Cleveland', '20', 'Washington', 'H', '', 'Western', '', 'W', '', '235', '94', '36', '79', '7', '20', '15', '22', '54', '22', '7', '5', '18', '12', '21']


In [ ]:
import datetime

def readTime(ampm):
  t = 0
  tL = ampm.replace(' ',':').split(':')
  if tL[2] == "PM" and int(tL[0]) < 12:
    t += 12
  t += int(tL[0])
  t += int(tL[1])/60
  return t

seasonBegin = datetime.datetime(2012, 10, 30)
noonTime = readTime("12:00 PM")
#not using
class Stats:
  def __init__(self):
    """
    MIN, PTS,FGM, FGA, 3FM, 3FA, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF
    """
    self.MIN = 0
    self.PTS = 0
    self.FGM = 0
    self.FGA = 0
    self.FM3 = 0
    self.FA3 = 0
    self.FTM = 0
    self.FTA = 0
    self.REB = 0
    self.AST = 0
    self.STL = 0
    self.BLK = 0
    self.OREB = 0
    self.TO = 0
    self.PF = 0
    self.Win = 0
    self.Loss = 0
    self.WinRate = 0
  def update(self, MIN, PTS,FGM, FGA, FM3, FA3, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF):
    self.MIN = MIN
    self.PTS = PTS
    self.FGM = FGM
    self.FGA = FGA
    self.FM3 = FM3
    self.FA3 = FA3
    self.FTM = FTM
    self.FTA = FTA
    self.REB = REB
    self.AST = AST
    self.STL = STL
    self.BLK = BLK
    self.OREB = OREB
    self.TO = TO
    self.PF = PF
  def addWinRate(self, Win, Loss):
    self.Win = Win
    self.Loss = Loss
    self.WinRate = Win/(Win+Loss)
  def notEmpty(self):
    return MIN>0

class aGame:
  def __init__(self, GameDay, GameID, TeamID, Team, OpponentID, Opponent, Location,
       Division, Conference, Playoffs, WinOrLoss, PlayerType):
    self.GameID = int(GameID)
    self.TeamID = int(TeamID)
    gdL = GameDay.split("-")
    self.GameDay = datetime.datetime(int(gdL[0]), int(gdL[1]), int(gdL[2]))
    #self.GameTime = GameTime
    self.DayCount = (self.GameDay - seasonBegin).days
    self.Team = Team
    self.OpponentID = int(OpponentID)
    self.Opponent = Opponent
    self.Home = Location == 'H'
    self.Division = Division
    self.Conference = Conference
    self.Playoffs = Playoffs == 'Y'
    self.WinOrLoss = WinOrLoss == 'W'
    self.WinID = int(self.WinOrLoss)
    self.LossID = int(not self.WinOrLoss)
    self.PlayerType = PlayerType
    self.TStats = Stats()
    self.OppStats = Stats()
    self.TStatsP = Stats()
    self.OppStatsP = Stats()
  def printDay(self):
    print('Day', self.DayCount)

def aLinetoGame(aline):
  newG = aGame(aline[0], aline[1], aline[2], aline[3], aline[4], aline[5], aline[6],
               aline[7], aline[8], aline[9], aline[10], aline[11])
  for n in range(12, 27):
    aline[n] = int(aline[n])

  newG.TStats.update(aline[12], aline[13], aline[14], aline[15], aline[16], aline[17], 
                     aline[18], aline[19], aline[20], aline[21], aline[22], aline[23], 
                     aline[24], aline[25], aline[26])
  return newG

def readLines(filename):
  with open(filename, newline='') as f:
      reader = csv.reader(f)
      next(reader)
      rawDataL = list(reader)
  #print(rawDataL)
  #'ID', 'GameDay', 'GameTime', 'Home', 'HomeID', 'Visitor', 'VisitorID', 'ScoreVis', 
  #'ScoreHome', 'GameStatus', 'Venue', 'CityState', 'Neutral', 'Division', 'Conference', 'Playoffs'
  #print("RawDataLength ",len(rawDataL))
  data = []
  for oneLine in rawDataL:
    g = aLinetoGame(oneLine)
    data.append(g)
  return data

In [ ]:
from copy import deepcopy
allData = readLines(filename)
print(allData[1])
print(len(allData))

for tg in allData:
  previousGamesList = list(filter(lambda g: (g.DayCount < tg.DayCount) and (g.TeamID == tg.TeamID), allData))
  pMIN = 0
  pPTS = 0
  pFGM = 0
  pFGA = 0
  pFM3 = 0
  pFA3 = 0
  pFTM = 0
  pFTA = 0
  pREB = 0
  pAST = 0
  pSTL = 0
  pBLK = 0
  pOREB = 0
  pTO = 0
  pPF = 0
  pWin = 0
  pLoss = 0
  for p in previousGamesList:
    g = p.TStats
    pMIN += g.MIN
    pPTS += g.PTS
    pFGM += g.FGM
    pFGA += g.FGA
    pFM3 += g.FM3
    pFA3 += g.FA3
    pFTM += g.FTM
    pFTA += g.FTA
    pREB += g.REB
    pAST += g.AST
    pSTL += g.STL
    pBLK += g.BLK
    pOREB += g.OREB
    pTO += g.TO
    pPF += g.PF
    pWin += p.WinID
    pLoss += p.LossID
  gPlayed = len(previousGamesList)
  if gPlayed > 0:
    tg.TStatsP.update(pMIN/gPlayed, pPTS/gPlayed, pFGM/gPlayed, pFGA/gPlayed, pFM3/gPlayed,
                      pFA3/gPlayed, pFTM/gPlayed, pFTA/gPlayed, pREB/gPlayed, pAST/gPlayed,
                      pSTL/gPlayed, pBLK/gPlayed, pOREB/gPlayed, pTO/gPlayed, pPF/gPlayed)
    tg.TStatsP.addWinRate(pWin, pLoss)
  else:
    tg.TStatsP.update(234, 100, 44, 90, 10,
                      30, 20, 25, 45, 25,
                      7, 7, 11, 15, 22)
    tg.TStatsP.addWinRate(1, 1)
    #Random for the first game
allGames = []
for d in allData:
  sameGameList = list(filter(lambda g: g.GameID == d.GameID, allGames))
  if len(sameGameList) > 0:
    for g in sameGameList:
      if g.Home: 
        g.OppStats = deepcopy(d.TStats)
        g.OppStatsP = deepcopy(d.TStatsP)
      else:
        d.OppStats = deepcopy(g.TStats)
        d.OppStatsP = deepcopy(g.TStatsP)
        allGames.remove(g)
        allGames.append(d)
  else:
    allGames.append(d)
allData = allGames
print(len(allData))

2628
1314


In [ ]:
def pick2Groups(allData):
  afterN = 120
  allData = list(filter(lambda g:not g.Playoffs, allData))
  nData = len(allData)
  test_daysAfterN = list(filter(lambda g: g.DayCount >= afterN, allData))
  test_daysAfterN = list(map(lambda g: g.DayCount, test_daysAfterN))
  test_days = list(set(test_daysAfterN))

  test_listGroups = []
  train_listGroups = []
  for daysC in test_days:
    test_listGroups.append(list(filter(lambda g: g.DayCount == daysC, allData)))
    train_listGroups.append(list(filter(lambda g: g.DayCount < daysC, allData)))
  return (test_listGroups, train_listGroups)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def gametoTensor(g : aGame, last_day):
  tensor = torch.zeros(1, 1, 78)
  
  tensor[0][0][g.TeamID] = 1
  tensor[0][0][g.OpponentID+37] = 1
  tensor[0][0][76] = last_day/300
  tensor[0][0][77] = g.DayCount/300
  
  
  return tensor

def gamestoTensor(games, test = False):
  g_tensor_list = []
  last_day = games[-1].DayCount + int(not test)
  for g in games:
    g_tensor_list.append((gametoTensor(g, last_day), g.WinID))
  return g_tensor_list

In [ ]:
import time
import math
from statistics import mean
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def runModel(test_listGroups, train_listGroups):
  start = time.time()
  test_tlistGroups = list(map(lambda group: gamestoTensor(group, test = True), test_listGroups))
  train_tlistGroups = list(map(lambda group: gamestoTensor(group), train_listGroups))
  totalC_list = []
  totalG_list = []
  print(len(train_tlistGroups), len(test_tlistGroups))
  for t in range(len(train_tlistGroups)):
    print('Model: ', t)
    model = nn.Sequential(nn.Linear(78, 64),
                          nn.Tanh(),
                          nn.Linear(64, 16),
                          nn.Tanh(),
                          nn.Linear(16, 2),
                          nn.LogSoftmax(dim=1))
    #model = model.to(device=device)
    learning_rate = 0.005
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_fn = nn.NLLLoss()#meansquarerror
    n_epochs = 30
    for epoch in range(n_epochs):
      
      train_loader = torch.utils.data.DataLoader(train_tlistGroups[t], batch_size = 4, shuffle = True)
      for gs, labels in train_loader:
        #gs = gs.to(device=device)
        #labels = labels.to(device = device)

        batch_size = gs.shape[0]
        outputs = model(gs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        """
      if (epoch+1)%20 == 0:
        print(timeSince(start))
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
        """
    totalC = 0
    totalG = 0
    for test_1day in test_tlistGroups[t]:
      gs, label = test_1day
      #gs = gs.to(device=device)
      outputs = model(gs.view(1, -1))
      _, predicted = torch.max(outputs, dim=1)
      totalG += 1
      totalC += int(predicted == label)
    totalC_list.append(totalC)
    totalG_list.append(totalG)
    accu = totalC / totalG
    print("Accuracy: %f" % (accu))
  accu = sum(totalC_list)/sum(totalG_list)
  print ("Overall Accuracy: %f" %(accu))
  print(timeSince(start))
  return accu

In [ ]:
def writeIn(newRecord):
  p1Name='/content/gdrive/My Drive/2021 Fall/Thesis I/Records/12_13AllTeamsS13.txt'

  p1file = open(p1Name)


  p1content = p1file.read()

  #p1file.close()

  p1file_content = p1file.read()
  p1file.seek(0)
  #Set file’s current position to the beginning
  p1file=open(p1Name,"w")
  p1file.write(p1content+str(newRecord)+"\n")
  p1file=open(p1Name)
  p1file.seek(0)
  p1file_content = p1file.read()
  p1file.close()

  print(p1file_content)

In [ ]:
trainGs, testGs = pick2Groups(allData)
for i in range(50):
  print('Run %d' %i)
  writeIn(runModel(trainGs, testGs))

Streaming output truncated to the last 5000 lines.
0.6675461741424802
0.6675461741424802
0.6728232189973615
0.6701846965699209

Run 13
49 49
Model:  0
Accuracy: 0.500000
Model:  1
Accuracy: 0.750000
Model:  2
Accuracy: 0.571429
Model:  3
Accuracy: 0.666667
Model:  4
Accuracy: 0.800000
Model:  5
Accuracy: 0.625000
Model:  6
Accuracy: 0.727273
Model:  7
Accuracy: 0.500000
Model:  8
Accuracy: 0.800000
Model:  9
Accuracy: 1.000000
Model:  10
Accuracy: 0.750000
Model:  11
Accuracy: 0.727273
Model:  12
Accuracy: 0.500000
Model:  13
Accuracy: 0.727273
Model:  14
Accuracy: 0.666667
Model:  15
Accuracy: 0.636364
Model:  16
Accuracy: 0.857143
Model:  17
Accuracy: 0.750000
Model:  18
Accuracy: 0.666667
Model:  19
Accuracy: 0.000000
Model:  20
Accuracy: 0.642857
Model:  21
Accuracy: 0.333333
Model:  22
Accuracy: 0.545455
Model:  23
Accuracy: 0.800000
Model:  24
Accuracy: 0.800000
Model:  25
Accuracy: 0.750000
Model:  26
Accuracy: 0.333333
Model:  27
Accuracy: 0.750000
Model:  28
Accuracy: 1.000000